In [1]:
import pandas as pd
import numpy as np
import json

In [96]:
# parse country codes to names, country codes taken from: https://datahub.io/core/country-list
with open("../dataset/country_codes.json", "r") as f:
    country_codes = json.load(f)
    code_to_country = {entry["Code"]: entry["Name"] for entry in country_codes}
    # "uk" and "el" were missing in the country codes list
    code_to_country["UK"] = "Utd. Kingdom"
    code_to_country["EL"] = "Greece"

In [15]:
# load the dataset of student exchanges
df = pd.read_csv("../dataset/erasmus-only-study-exchange.csv")

In [11]:
# translate existing field of study entries to categories of larger granularity
# mapping has been done manually, see field_of_study_mapping.csv for details
field_of_study_mapping = {
1: "Natural Sciences and Mathematics",
2: "Engineering and Technology",
3: "Humanities, Arts and Social Science",
4: "Medicine and Health",
5: "Economics and Business",
6: "Languages",
7: "Other",
8: "Not specified"}
foe_labels = pd.read_csv("../dataset/field_of_study_mapping.csv", sep="; ")

df = pd.merge(df, foe_labels, on="Field of Education")
df["Field of Education"] = df["Label"].apply(lambda x: field_of_study_mapping[x])

/Users/ndresevic/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


In [7]:
df["Field of Education"].value_counts()

Economics and Business                 281383
Humanities, Arts and Social Science    279115
Engineering and Technology             190865
Languages                              138740
Other                                  102934
Medicine and Health                     65730
Natural Sciences and Mathematics        49649
Not specified                           13949
Name: Field of Education, dtype: int64

In [109]:
# prepare dataset for visualization 1: students per country (with attributes: gender, field of education, sending/receiving, academic year)
# we want the following tabular dataset:
# | Country | Year | University | Direction | Gender | Field | Participants |
# ===========================================================================
# | NL      | 2018 | TU Delft   | Sending   | Male   | CS    | 8            |

df_female = df[df["Participant Gender"] == "Female"]
df_male = df[df["Participant Gender"] == "Male"]
grouping_columns = ['Academic Year']

# SENDING FEMALE
sending_female = df_female[grouping_columns + ["Participants", "Sending Country Code"]]
sending_female = sending_female.groupby(["Academic Year", "Sending Country Code"]).agg({"Participants": "sum"}).reset_index()
sending_female.rename(columns={"Sending Country Code": "Country", "Participants" : "Sending-Female"}, inplace=True)

# SENDING MALE
sending_male = df_male[grouping_columns + ["Participants", "Sending Country Code"]]
sending_male = sending_male.groupby(["Academic Year", "Sending Country Code"]).agg({"Participants": "sum"}).reset_index()
sending_male.rename(columns={"Sending Country Code": "Country", "Participants" : "Sending-Male"}, inplace=True)

# SENDING ALL
sending_all = df[grouping_columns + ["Participants", "Sending Country Code"]]
sending_all = sending_all.groupby(grouping_columns + ["Sending Country Code"]).agg({"Participants": "sum"}).reset_index()
sending_all.rename(columns={"Sending Country Code": "Country", "Participants" : "Sending-All"}, inplace=True)

# RECEIVING FEMALE
receiving_female = df_female[grouping_columns + ["Participants", "Receiving Country Code"]]
receiving_female = receiving_female.groupby(["Academic Year", "Receiving Country Code"]).agg({"Participants": "sum"}).reset_index()
receiving_female.rename(columns={"Receiving Country Code": "Country", "Participants" : "Receiving-Female"}, inplace=True)

# RECEIVING MALE
receiving_male = df_male[grouping_columns + ["Participants", "Receiving Country Code"]]
receiving_male = receiving_male.groupby(["Academic Year", "Receiving Country Code"]).agg({"Participants": "sum"}).reset_index()
receiving_male.rename(columns={"Receiving Country Code": "Country", "Participants" : "Receiving-Male"}, inplace=True)

# RECEIVING ALL
receiving_all = df[grouping_columns + ["Participants", "Receiving Country Code"]]
receiving_all = receiving_all.groupby(grouping_columns + ["Receiving Country Code"]).agg({"Participants": "sum"}).reset_index()
receiving_all.rename(columns={"Receiving Country Code": "Country", "Participants" : "Receiving-All"}, inplace=True)

sending_merge = pd.merge(sending_female, sending_male, on=['Academic Year', 'Country'],how='outer').reset_index(drop=True)
sending_merge = pd.merge(sending_merge, sending_all, on=['Academic Year', 'Country'],how='outer').reset_index(drop=True)
receiving_merge = pd.merge(receiving_female, receiving_male, on=['Academic Year', 'Country'],how='outer').reset_index(drop=True)
receiving_merge = pd.merge(receiving_merge, receiving_all, on=['Academic Year', 'Country'],how='outer').reset_index(drop=True)
visualization_1 = pd.merge(sending_merge, receiving_merge, on=['Academic Year', 'Country'],how='outer').reset_index(drop=True)
visualization_1["All-Female"] = visualization_1["Sending-Female"] + visualization_1["Receiving-Female"]
visualization_1["All-Male"] = visualization_1["Sending-Male"] + visualization_1["Receiving-Male"]
visualization_1["All-All"] = visualization_1["Sending-All"] + visualization_1["Receiving-All"]

# Map country id to name, and manually to match for world map
visualization_1["Country"].replace(code_to_country, inplace=True)
visualization_1["Country"].replace({"Czechia" : "Czech Republic", "Serbia" : "Republic of Serbia", "Utd. Kingdom" : "England"}, inplace=True)
visualization_1.head(10)

j = (visualization_1.groupby(['Country'])
       .apply(lambda x: x[['Academic Year','Sending-Female', 'Sending-Male', 'Sending-All', 'Receiving-Female', 'Receiving-Male', 'Receiving-All', 'All-Female', 'All-Male', 'All-All']].to_dict('records'))
       .reset_index()
       .rename(columns={0:'Yearly-Data'})
       .to_json('../dataset/viz1.json', indent=2, orient='records'))

In [108]:
visualization_1

,Academic Year,Country,Sending-Female,Sending-Male,Sending-All,Receiving-Female,Receiving-Male,Receiving-All,All-Female,All-Male,All-All
0,2014-2015,Austria,1838.0,1049.0,2887.0,1996,1343,3339,3834.0,2392.0,6226.0
1,2014-2015,Belgium,2743.0,1584.0,4327.0,2855,1537,4392,5598.0,3121.0,8719.0
2,2014-2015,Bulgaria,478.0,203.0,681.0,240,241,481,718.0,444.0,1162.0
3,2014-2015,Cyprus,104.0,58.0,162.0,199,133,332,303.0,191.0,494.0
4,2014-2015,Czech Republic,2550.0,1511.0,4061.0,2112,2118,4230,4662.0,3629.0,8291.0
...,...,...,...,...,...,...,...,...,...,...,...
194,2019-2020,Slovenia,752.0,450.0,1202.0,1069,732,1801,1821.0,1182.0,3003.0
195,2019-2020,Slovakia,1166.0,633.0,1799.0,644,654,1298,1810.0,1287.0,3097.0
196,2019-2020,Turkey,6305.0,6300.0,12605.0,1400,1024,2424,7705.0,7324.0,15029.0
197,2019-2020,England,4929.0,2766.0,7695.0,7894,4096,11990,12823.0,6862.0,19685.0


In [12]:
# visualization 2 - chord diagram - for each pair of countries (c1,c2) we want {from: c1, to: c2, participants: X}
visualization_2 = df[["Sending Country Code", "Receiving Country Code", "Participants"]] \
    .groupby(["Sending Country Code", "Receiving Country Code"]) \
    .agg({"Participants": "sum"}).sort_values("Participants", ascending=False).reset_index()

# fix the problem with missing row for RS
rs = {"Sending Country Code": 'RS', "Receiving Country Code": 'UK', "Participants": 0}
visualization_2 = visualization_2.append(rs, ignore_index=True)

visualization_2.to_csv("../dataset/visualization_2.csv")

/var/folders/by/p988t6fx4wqg62rr_dqt5r_80000gn/T/ipykernel_36736/1608807140.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualization_2 = visualization_2.append(rs, ignore_index=True)


In [13]:
visualization_2

,Sending Country Code,Receiving Country Code,Participants
0,IT,ES,48445
1,ES,IT,34643
2,DE,ES,27863
3,FR,ES,27188
4,DE,FR,24358
...,...,...,...
982,MT,BG,1
983,AT,MK,1
984,LI,SI,1
985,IS,SK,1


In [36]:
# we want to find the total number of participnants for each country
visualization_2[["Sending Country Code", "Participants"]].groupby("Sending Country Code").agg({"Participants": "sum"}).sort_values("Participants", ascending=False)

# we want to find the total number of participants for each country and select only top 10 countries for the visualization
sent = visualization_2[["Sending Country Code", "Participants"]].groupby("Sending Country Code").agg(
    {"Participants": "sum"}).sort_values("Participants", ascending=False).rename(columns = {"Participants": "Sent"})

received = visualization_2[["Receiving Country Code", "Participants"]].groupby("Receiving Country Code").agg(
    {"Participants": "sum"}).sort_values("Participants", ascending=False).rename(columns = {"Participants": "Received"})

counts = sent.join(received)
counts["Total"] = counts["Sent"] + counts["Received"]
counts = counts.sort_values("Total", ascending=False)
top_countries = counts.index.values[:10]
counts, top_countries

(                        Sent  Received   Total
 Sending Country Code                          
 ES                    161287    175506  336793
 DE                    165080    116817  281897
 FR                    154772    116244  271016
 IT                    154218    100368  254586
 UK                     51031     93660  144691
 PL                     56758     71503  128261
 NL                     48266     49978   98244
 PT                     34457     57436   91893
 TR                     70467     20077   90544
 CZ                     31470     38895   70365
 BE                     34976     34549   69525
 SE                     18504     41454   59958
 FI                     22523     36896   59419
 AT                     19020     27900   46920
 IE                     14124     25900   40024
 NO                     11139     28070   39209
 DK                     15419     21669   37088
 HU                     11849     23869   35718
 RO                     20330     12688 

In [37]:
visualization_2 = visualization_2[visualization_2["Sending Country Code"].isin(top_countries)]
visualization_2 = visualization_2[visualization_2["Receiving Country Code"].isin(top_countries)]
visualization_2

,Sending Country Code,Receiving Country Code,Participants
0,IT,ES,48445
1,ES,IT,34643
2,DE,ES,27863
3,FR,ES,27188
4,DE,FR,24358
...,...,...,...
279,NL,PL,752
305,CZ,TR,644
395,UK,PL,379
414,PT,TR,333


In [39]:
visualization_2_matrix = visualization_2.pivot(index=["Sending Country Code"], columns=["Receiving Country Code"], values=["Participants"]).fillna(0)

In [44]:
matrix_values = visualization_2_matrix.values
matrix_labels = [code_to_country[code] for code in visualization_2_matrix.index.values]

In [45]:
# save the data to json so it can be read in the visualization
import json

with open("../docs/data/viz2.json", "w+") as f:
    data = {"matrix": matrix_values.tolist(),
    "labels": matrix_labels}
    json.dump(data, f)

In [42]:
# visualization 3: popularity of fields per year
visualization_3 = df[["Academic Year", "Field of Education", "Participants"]] \
    .groupby(["Academic Year", "Field of Education"]) \
    .agg({"Participants": "sum"}).sort_values(["Academic Year", "Field of Education"]).reset_index()
visualization_3.to_csv("../dataset/visualization_3.csv")
visualization_3

,Academic Year,Field of Education,Participants
0,2014-2015,Economics and Business,35818
1,2014-2015,Engineering and Technology,23552
2,2014-2015,"Humanities, Arts and Social Science",37220
3,2014-2015,Languages,20361
4,2014-2015,Medicine and Health,7600
5,2014-2015,Natural Sciences and Mathematics,6431
6,2014-2015,Not specified,402
7,2014-2015,Other,10683
8,2015-2016,Economics and Business,53693
9,2015-2016,Engineering and Technology,35840


In [23]:
# Visualization 4: bubble - bubble sizes based on total participants or pageranks
exchanges = df[["Sending Organization", "Receiving Organization", "Participants"]]
links = exchanges.groupby(["Sending Organization", "Receiving Organization"]).agg('sum').reset_index().sort_values("Participants", ascending=False) \
    .rename(columns={"Sending Organization":"from", "Receiving Organization":"to", "Participants": "links"}).reset_index(drop=True)

In [28]:
import networkx as nx
# create a graph
G = nx.from_pandas_edgelist(links, source="from", target="to", edge_attr="links", create_using=nx.DiGraph())

In [33]:
pagerank = nx.pagerank(G)

In [36]:
betweenness = nx.betweenness_centrality(G)

In [37]:
betweenness

{'ALMA MATER STUDIORUM - UNIVERSITA DI BOLOGNA': 0.01730079214175424,
 'UNIVERSITE PARIS OUEST NANTERRE LA DEFENSE': 0.00039539225270994417,
 'UNIVERSITEIT NYENRODE BV': 4.877650874044065e-08,
 'THE UNIVERSITY OF WESTMINSTER LBG': 0.00011510153107460185,
 'UNIVERSITAET ZU KOELN': 0.001224269122166702,
 'UNIVERSITE PARIS I PANTHEON-SORBONNE': 0.00036717791197358644,
 'Groupe KEDGE Business School': 0.0003535635322337763,
 'UNIVERSITY OF HULL': 0.0003383921186296174,
 'JOHANNES GUTENBERG-UNIVERSITAT MAINZ': 0.0032534095724693407,
 'UNIVERSITE DIJON BOURGOGNE': 0.0006410924853657758,
 'UNIVERSIDAD DE GRANADA': 0.023197583000153333,
 'A.E.S.C.R.A.': 2.6457601367050006e-05,
 'LUDWIG-MAXIMILIANS-UNIVERSITAET MUENCHEN': 0.001803191746603927,
 'UNIVERSITE DE LORRAINE': 0.003325136469404259,
 'LULEA TEKNISKA UNIVERSITET': 0.00023412116226700127,
 'POLITECNICO DI MILANO': 0.009925387560218157,
 'UNIVERSITAT POLITECNICA DE CATALUNYA': 0.0075844186891525535,
 'UNIVERSITAT DE VALENCIA': 0.008457202

In [63]:
# calculate total number of incoming/outgoing students per university
outgoing = links[["from", "links"]].groupby("from").agg("sum").sort_values("links", ascending=False).reset_index().rename(columns={"from": "university", "links":"outgoing"})
incoming = links[["to", "links"]].groupby("to").agg("sum").sort_values("links", ascending=False).reset_index().rename(columns={"to": "university", "links":"incoming"})

In [84]:
# add rankings 
ranking_df = pd.merge(outgoing, incoming, on="university", how="outer").fillna(0)
ranking_df["total"] = ranking_df["incoming"] + ranking_df["outgoing"]
ranking_df["pagerank"] = ranking_df["university"].apply(lambda uni: pagerank[uni])
ranking_df["betweenness"] = ranking_df["university"].apply(lambda uni: betweenness[uni])


In [85]:
ranking_df = ranking_df.sort_values("total", ascending=False).reset_index(drop=True)


In [86]:
ranking_df

,university,outgoing,incoming,total,pagerank,betweenness
0,ALMA MATER STUDIORUM - UNIVERSITA DI BOLOGNA,12184.0,10148.0,22332.0,0.003202,0.017301
1,UNIVERSIDAD DE GRANADA,7366.0,9773.0,17139.0,0.003434,0.023198
2,UNIVERSIDAD COMPLUTENSE DE MADRID,7628.0,8701.0,16329.0,0.003461,0.018874
3,UNIVERSITAT DE VALENCIA,5753.0,10386.0,16139.0,0.002739,0.008457
4,UNIVERSIDADE DE LISBOA,5944.0,9514.0,15458.0,0.003691,0.016317
...,...,...,...,...,...,...
3102,Institut de Formation en Soins Infirmiers du C...,0.0,1.0,1.0,0.000061,0.000000
3103,Wyzsza Szkola Inzynierii i Zdrowia w Warszawie,0.0,1.0,1.0,0.000058,0.000000
3104,Ewangelikalna Wyzsza Szkola Teologiczna,1.0,0.0,1.0,0.000056,0.000000
3105,Latvian Business College,1.0,0.0,1.0,0.000056,0.000000


In [87]:
ranking_df.to_csv("../dataset/visualization_4.csv")